In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import cv2

In [ ]:
IMAGES_PATH = "../SignTracker/hand_images"

In [ ]:
label = os.listdir(IMAGES_PATH) #solo 'hand'
print(labels)

In [ ]:
image_array=[]

In [ ]:
images = os.listdir(IMAGES_PATH + "/" + labels)
    #print(images)
    
    for j in range(len(images)):
        file_path = IMAGES_PATH + "/" + labels + "/" + images[j]
        
        #read
        img = cv2.imread(file_path)
        #resize
        img = cv2.resize(img,(300,300))
        #convert BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #add img
        image_array.append(img)
        #add label
        label_array.append(i)
        

In [ ]:
#list to array
image_array = np.array(image_array)
label_array = np.array(label_array, dtype = "float")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image_array, label_array, test_size = 0.15)

# x_train will have 85% of images 
# x_test will have 15% of images

In [ ]:
del image_array

In [ ]:
#memory-free
import gc
gc.collect()

In [ ]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential, Model, load_model

In [ ]:
model = Sequential()

In [ ]:
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape = (300, 300, 3), include_top = False)
model.add(pretrained_model)

In [ ]:
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1))
model.build(input_shape=(None,300,300,3))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

In [ ]:
check_path = "../SignTracker/models/hand_checkpoint"

In [ ]:
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath = check_path,
    monitor = "val_mae",
    mode = "auto",
    save_best_only = True,
    save_weights_only = True
)

In [ ]:
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
    factor = 0.9, #lr= 0.9* current lr
    monitor = "val_mae",
    mode = "auto",
    cooldown = 0,
    patience = 5,
    verbose = 1,
    min_le = 1e-6
)

In [ ]:
history=model.fit(
    x_train,
    y_train,
    validation_data = (x_test, y_test),
    batch_size = 16,
    epochs = 150,
    callbacks = [model_checkpoint, reduce_lr]
)

In [ ]:
model.load_weights(check_path)

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

In [ ]:
with open("HandTrackerModel.tflite","wb") as f:
    f.write(tflite_model)

In [ ]:
prediction_val=model.predict(x_test,batch_size=32)

In [ ]:
print(prediction_val[:30])
print(y_test[:30])